1.  data_sent_new.csv로 대분류 감성 분류 모형을 개발 (model1)
- 모형 튜닝 사용
- 파티셔닝을 통한 성능 파악
- vect1 : Vectorizer,

2.  seoul-120-text.csv로 질문 분류 컬럼에 대한 분류 모형 (model2)
- 모형 튜닝 사용
- 파티셔닝을 통한 성능 파악
- vect2 :  Vectorizer,

3. train_df의 질문 내용에 대해서 1과 2의 모형 예측을 각각 수행
- dtm = vect1.transform(텍스트 )
- dtm = vect2.transform(텍스트 )
- 00개에 대해서만 적용
- 데이터프레임에 예측 결과를 컬럼으로 추가

4. 3에서 부정적인 질문(또는 특정 범주의 질문들)으로 부터 토픽 발견
- 최적의 토픽 개수 탐색
- 해석

5. 발견된 토픽에 대해서, 관련성이 높은 키워드로 뉴스를 수집하여 토픽 모델링
- 최적의 토픽 개수
- 해석

6. 1~5에 대해서 보고서 스타일로 결과 정리
- gpt로 초안 ppt 생성
- 모형에 대한 설명, 분석 결과 해석 등을 추가


In [3]:
!pip install konlpy
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.1 MB/s eta 0:00:00


In [25]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

###1번

In [6]:
df1 = pd.read_csv("data_sent_new.csv", encoding="cp949")
df1.head()

,연령,감정_대분류,감정_소분류,sentence
0,청년,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점...
1,노년,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 건강할 때 ...
2,청소년,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 아직 학기 ...
3,노년,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. ...
4,노년,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 빚도 다 갚았으니 당분간은 아무...


In [28]:
df1.감정_대분류.value_counts()
df1_2 = df1.query("(감정_대분류=='불안') or (감정_대분류=='당황')")
df1_2 = df1_2.sample(2000)

In [29]:
okt = Okt()
def token(text):
    return okt.nouns(text)

vect1 = CountVectorizer(tokenizer=token)
dtm1 = vect1.fit_transform(df1_2["sentence"])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [37]:
X_train1, X_test1, y_train1, y_test1 = train_test_split( dtm1, df1_2.감정_대분류)
dt1 = DecisionTreeClassifier()
dt1.fit(X_train1, y_train1)
pred1 = dt1.predict(X_test1)
print(classification_report(y_test1, pred1))


              precision    recall  f1-score   support

          당황       0.62      0.64      0.63       230
          불안       0.69      0.67      0.68       270

    accuracy                           0.66       500
   macro avg       0.65      0.66      0.65       500
weighted avg       0.66      0.66      0.66       500



###2번

In [32]:
df2 = pd.read_csv("seoul-120-text.csv")
df2.head()

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경제적 부담을 완화함으...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플...,23194045
2,2643,환경,(강북구) 정비중,업무개요 투명 폐트병을 교환 보상하므로 수거율을 높이고 폐기물을 감량하여 환경 보...,23032485
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄 쾌적하고 안...,22904492
4,2641,복지,마포 뇌병변장애인 비전센터,마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈 곳이 없는...,22477798


In [34]:
df2.분류.value_counts()
df2_2 = df2.query("(분류=='복지') or (분류=='경제')")
df2_2 = df2_2.sample(1000)

In [35]:
okt = Okt()
def token(text):
    return okt.nouns(text)

vect2 = CountVectorizer(tokenizer=token)
dtm2 = vect2.fit_transform(df2_2["내용"])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
X_train2, X_test2, y_train2, y_test2 = train_test_split( dtm2, df2_2.분류)
dt2 = DecisionTreeClassifier()
dt2.fit(X_train2, y_train2)
pred2 = dt2.predict(X_test2)
print(classification_report(y_test2, pred2))


              precision    recall  f1-score   support

          경제       0.91      0.95      0.93       191
          복지       0.80      0.69      0.75        59

    accuracy                           0.89       250
   macro avg       0.86      0.82      0.84       250
weighted avg       0.88      0.89      0.89       250



###3번

In [40]:
df3 = pd.read_csv("train_df.csv", encoding="cp949")
df3.head()

,IDX,발화자,발화문,인텐트,상담번호,상담내순번
0,28414,c,혹시 플라워 스마트 톡 케이스 블루로 바꿀 수 있나요?,교환|반품|환불_일반_요청,389321,1
1,28415,s,현재 배송 중으로 교환이 어렵습니다.,교환|반품|환불_일반_요청,389321,2
2,28416,c,USB1.0 속도 밖에 나오질 않네요리더 기만 교환 바랍니다. 불량 리더기도 택배로...,교환|반품|환불_일반_요청,179614,1
3,28417,s,네.리더기만 불량이 여교환 문의해주셨는데요.,교환|반품|환불_일반_요청,179614,2
4,28418,c,빨리 챙겨서 보내주시기 바랍니다 아니면 다시 1개는 교환으로 보내야 하는지요?,교환|반품|환불_일반_요청,325053,1


In [41]:
df3_1 = df3.query("발화자=='s'").sample(50)

In [44]:
dtm3_1 = vect1.transform(df3_1["발화문"])
dtm3_2 = vect2.transform(df3_1["발화문"])

In [46]:
pred3_1 = dt1.predict(dtm3_1)
pred3_2 = dt2.predict(dtm3_2)

In [47]:
pred3_1

array(['불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '당황', '불안', '불안',
       '불안', '불안', '당황', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안',
       '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안',
       '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안', '불안',
       '불안', '불안', '불안', '불안', '당황', '불안'], dtype=object)

In [48]:
pred3_2

array(['경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제',
       '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제',
       '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제',
       '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제', '경제',
       '경제', '경제', '경제', '경제', '복지', '경제'], dtype=object)

In [51]:
df3_1["모형1예측"] = pred3_1
df3_1["모형2예측"] = pred3_2
df3_1.head()

,IDX,발화자,발화문,인텐트,상담번호,상담내순번,모형1예측,모형2예측
15014,539675,s,"네, 환불 신청서 작성 부탁드리겠습니다.",교환|반품|환불_방법_요청,81905,2,불안,경제
21176,648709,s,반송하신 상품 아직 업체 입고 중으로 확인됩니다. 입고 확인 후 환불 처리 진행하겠...,교환|반품|환불_시간_질문,322344,2,불안,경제
3376,106243,s,기존 주문건환 불완료 진행되었으며 본사 발주건은 4/2 설치 예정으로 조회되어 기존...,교환|반품|환불_일반_질문,191548,2,불안,경제
22456,649989,s,제품 수거가 아직 진행되지 않은 상태입니다. 담당 부서 입고 확인되는 대로 처리해드...,교환|반품|환불_시간_질문,148661,2,불안,경제
153,28567,s,고객님의 제품은 금일 재발송 처리 처리 및 안내드린 대로 수거 진행하도록 하겠습니다.,교환|반품|환불_일반_요청,197708,2,불안,경제


###4번

In [55]:
!pip install tqdm #진행상태를 알려주는 기능 제공

In [79]:
from gensim import corpora
from tqdm import tqdm
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import gensim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
df4 = df3_1.query("모형1예측=='불안'")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
train_data = df4.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data['발화문'] = train_data['발화문'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z ]","", regex=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [82]:
#값을 저장할 비어있는 리스트를 만듬.
tokenized_data4 = []

# 불용어 정의
stopwords = ['을', 'ㅋㅋ',
             '부터', '까지' '적', '의','가','이','은',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

for sentence in tqdm(train_data['발화문']): #리뷰 각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 리뷰에서 명사만 뽑아서 리스트
    #위에서 명사만 추출한 결과에서, 각 명사가 stopwords인지 체크하고
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data4.append(stopwords_removed_sentence)

len(tokenized_data4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 47/47 [00:00<00:00, 268.94it/s]


47

In [83]:
dictionary4 = corpora.Dictionary(tokenized_data4) #사전 구성
corpus4 = [dictionary4.doc2bow(text) for text in tokenized_data4]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
NUM_TOPICS = 3
ldamodel4 = gensim.models.ldamodel.LdaModel(corpus4, num_topics = NUM_TOPICS,
                                           id2word=dictionary4, passes=15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [85]:
#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics4 = ldamodel4.print_topics(num_words=15)
#위 내용을 출력
for topic in topics4:
    print(topic)

(0, '0.051*"반품" + 0.050*"제품" + 0.050*"환불" + 0.037*"처리" + 0.020*"교환" + 0.016*"고객" + 0.015*"주문" + 0.015*"결제" + 0.011*"해당" + 0.011*"문제" + 0.011*"금일" + 0.011*"충전" + 0.011*"때문" + 0.011*"및" + 0.011*"출고"')
(1, '0.043*"확인" + 0.037*"교환" + 0.031*"제품" + 0.023*"일" + 0.023*"택배" + 0.019*"처리" + 0.019*"접수" + 0.017*"경우" + 0.017*"사용" + 0.016*"주시" + 0.012*"중" + 0.012*"주문" + 0.012*"빛" + 0.012*"필름" + 0.012*"세탁기"')
(2, '0.040*"진행" + 0.038*"확인" + 0.032*"환불" + 0.025*"후" + 0.024*"상품" + 0.022*"처리" + 0.021*"교환" + 0.019*"접수" + 0.015*"건" + 0.015*"사진" + 0.015*"첨부" + 0.015*"것" + 0.015*"일" + 0.014*"저희" + 0.014*"쪽"')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [86]:
pyLDAvis.enable_notebook()
vis4 = pyLDAvis.gensim_models.prepare(ldamodel4, corpus4, dictionary4, sort_topics=True)
pyLDAvis.display(vis4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [87]:
coherence_values4 = []
model_list4 = []
iters = [3,4,5,6]
for num_topics in iters:
     model4 = LdaModel(corpus=corpus4, id2word=dictionary4, num_topics=num_topics)
     model_list4.append(model4)
     coherencemodel4 = CoherenceModel(model=model4, texts=tokenized_data4, dictionary=dictionary4, coherence='c_v')
     coherence_values4.append(coherencemodel4.get_coherence())

for i in range(0, len(iters)):
  print(model_list4[i], coherence_values4[i])  #topic num별로 coherence 가 높은 것을 찾기

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LdaModel<num_terms=199, num_topics=3, decay=0.5, chunksize=2000> 0.4172162910766753
LdaModel<num_terms=199, num_topics=4, decay=0.5, chunksize=2000> 0.44642768926585197
LdaModel<num_terms=199, num_topics=5, decay=0.5, chunksize=2000> 0.38398373312139267
LdaModel<num_terms=199, num_topics=6, decay=0.5, chunksize=2000> 0.3598216130381957


In [88]:
topics = model_list[0].get_document_topics(corpus)
df4['topics'] = topics
df4.topics.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-88-69706b9c63ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['topics'] = topics


,topics
15014,"[(0, 0.068093956), (1, 0.8630584), (2, 0.06884..."
21176,"[(0, 0.93036133), (1, 0.034251843), (2, 0.0353..."
3376,"[(0, 0.020191094), (1, 0.95935935), (2, 0.0204..."
22456,"[(0, 0.03704165), (1, 0.035928078), (2, 0.9270..."
153,"[(0, 0.9347803), (1, 0.03046328), (2, 0.034756..."


In [89]:
def get_max(doc):
    idx,l = zip(*doc)
    return idx[np.argmax(l)]

df4["topic"] = df4.topics.apply(get_max)
df4.topic.value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-89-a74fbaf46310>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["topic"] = df4.topics.apply(get_max)


,count
topic,
2,18
1,15
0,14


- 해석
 - (0): 진행, 처리, 사용 등의 단어는 아마도 제품 처리 과정이나 절차와 관련된 주제를 나타냅니다.
 - (1): 제품, 교환, 환불 등의 단어는 제품의 교환, 환불과 같은 고객 서비스 관련 문제를 다루는 주제를 반영하는 것 같습니다.
 - (2): 확인, 처리, 환불, 반품 등의 단어는 주문 후 제품 확인 및 처리 과정에 대한 주제를 나타내는 것처럼 보입니다.

###5번

In [90]:
from bs4 import BeautifulSoup
from lxml import etree
from urllib import parse
import requests
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
#----------------------------------------------#
#변경할 부분
startdate = "2024.04.01"
enddate = "2024.04.20"
searchword = "반품" #검색어
#----------------------------------------------#

base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query"+urllib.parse.urlencode([('',searchword)], doseq=True)+"&sort=1&photo=3&field=0&pd=3&ds="+startdate+"&de="+enddate+"&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from"+startdate.replace(".", "")+"to"+enddate.replace(".", "")+",a:all&mynews=0&start="
titles = [ ]
urls = [ ]
i=1

while True:
 try:
    time.sleep(0.15)
    url = base_url+str(i)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    t_news = soup.select('a.news_tit')  #클래스 표시
    t_len = len(t_news)
    print(t_news)

    if t_len <1:
        break

    for j in np.arange(t_len)+1:
            try:
                titles.append(soup.select('a.news_tit')[j].text)
                urls.append(soup.select('a.info')[j]['href'])
            except Exception as e:
                continue
    i=i+10

 except Exception as e:
    continue

result = pd.DataFrame({"Url": urls, "Title":titles})
result.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[<a class="news_tit" href="https://www.chosun.com/economy/tech_it/2024/04/19/QAZTQNZXEVBWFIHNJFLSQYQGFU/?utm_source=naver&amp;utm_medium=referral&amp;utm_campaign=naver-news" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=1&amp;i=880000ED_000000000000000003829267&amp;g=023.0003829267&amp;u='+urlencode(this.href));" target="_blank" title="네이버도 ‘당일 배송’ 불붙은 고객 쟁탈전">네이버도 ‘당일 배송’ 불붙은 고객 쟁탈전 </a>, <a class="news_tit" href="https://www.khan.co.kr/economy/economy-general/article/202404182151015" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=2&amp;i=8800006B_000000000000000003291557&amp;g=032.0003291557&amp;u='+urlencode(this.href));" target="_blank" title="네이버, 수도권 당일·일요배송…멤버십 인상 쿠팡 빈틈 공략">네이버, 수도권 당일·일요배송…멤버십 인상 쿠팡 빈틈 공략 </a>, <a class="news_tit" href="http://www.fnnews.com/news/202404181818161899" onclick="return goOtherCR(this, 'a=nws*j.tit&amp;r=3&amp;i=880000FF_000000000000000005173425&amp;g=014.0005173425&amp;u='+urlencode(this.href));" target="_blank" title="해외 사업자 규제 실효성 의문… 중

(46, 2)

 - 본문 수집

In [134]:
articles = []
dates = []

for i in result.Url:
   try:
    if( "news.naver.com" in i ):
     print(i)
     time.sleep(0.15)
     html = urllib.request.urlopen(i).read()
     soup = BeautifulSoup(html, 'html.parser')
     news = soup.select('article#dic_area')[0].text #태그 뒤 id는 #, class는 .
     news_date = soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text  #네이버 뉴스의 날짜 시간 부분 지정
     articles.append(news)
     dates.append(news_date.split()[0])
   except Exception as e:
    print(e)
    continue



https://n.news.naver.com/mnews/article/023/0003829267?sid=105


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


https://n.news.naver.com/mnews/article/032/0003291557?sid=101
https://n.news.naver.com/mnews/article/014/0005173425?sid=105
https://n.news.naver.com/mnews/article/015/0004974559?sid=101
https://n.news.naver.com/mnews/article/009/0005290552?sid=103
https://n.news.naver.com/mnews/article/023/0003828954?sid=101
https://n.news.naver.com/mnews/article/029/0002868269?sid=101
https://n.news.naver.com/mnews/article/014/0005172216?sid=110
https://n.news.naver.com/mnews/article/016/0002295562?sid=101
https://n.news.naver.com/mnews/article/018/0005716058?sid=101
https://n.news.naver.com/mnews/article/005/0001688572?sid=101
https://n.news.naver.com/mnews/article/082/0001265155?sid=101
https://n.news.naver.com/mnews/article/009/0005287949?sid=101
https://n.news.naver.com/mnews/article/030/0003197607?sid=101
https://n.news.naver.com/mnews/article/030/0003197609?sid=101
https://n.news.naver.com/mnews/article/023/0003828032?sid=101
https://n.news.naver.com/mnews/article/011/0004327037?sid=101
https://

In [135]:
# 수집된 내용을 표로 정리하기, news컬럼, date컬럼
df5 = pd.DataFrame({'news':articles, 'date':dates})
df5

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,news,date
0,"\n수도권만 대상, 내년 지역 확대\t\t\t\t\t\t\t네이버가 수도권에서 라면...",2024.04.19.
1,\n‘도착보장’ 앞세워 영토 확장…네이버플러스 이용자엔 3개월간 무료네이버가 물류 ...,2024.04.18.
2,\n'알테쉬' 개인정보 유출 우려국내법 위반 확인 쉽지 않지만글로벌 업체 처벌사례도...,2024.04.18.
3,\n수도권부터 도입…내년 확대일요배송·무료 반품 등 지원네이버가 물류 솔루션 ‘네이...,2024.04.18.
4,\n이커머스 멤버십 강화 '속도전'네이버가 전자상거래(이커머스) 물류 서비스인 '네...,2024.04.18.
5,\n\n\n\n\n서울 시내의 쿠팡 캠프에서 배송 기사들이 배송준비 작업을 하고 있...,2024.04.18.
6,"\n업계, 140만명 겨냥 정책 펼쳐국내 이커머스(전자상거래) 1위 업체인 쿠팡의 ...",2024.04.17.
7,"\n\n\n\n\n이석우 베이징특파원·대기자 ""18위안(3430원) 맞아요?"" 베이...",2024.04.16.
8,\n와우 회원에 월 8만원 혜택쿠플 콘텐츠·특가 행사도 강화‘8만원’. 쿠팡 ‘와우...,2024.04.16.
9,\n와우요금 4980원→7890원 인상에 ‘시끌’무료배달 추진하더니 결국 소비자 전...,2024.04.16.


- 전처리 및 명사 추출

In [136]:
df5_1 = df5.dropna(how = 'any') # Null 값이 존재하는 행 제거

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [137]:
df5_1['news'] = df5_1['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [138]:
#값을 저장할 비어있는 리스트를 만듬.
tokenized_data5 = []
# 불용어 정의
stopwords = ['을', 'ㅋㅋ',
             '부터', '까지' '적', '의','가','이','은',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나']
# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()
for sentence in tqdm(df5_1['news']): #각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 문헌에서 명사만 뽑아서 리스트
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence
                                  if not word in stopwords] # 불용어 제거
    tokenized_data5.append(stopwords_removed_sentence)

len(tokenized_data5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 26/26 [00:02<00:00, 11.07it/s]


26

In [139]:
dictionary5 = corpora.Dictionary(tokenized_data5) #사전 구성
corpus5 = [dictionary5.doc2bow(text) for text in tokenized_data5]

NUM_TOPICS = 3
ldamodel5 = gensim.models.ldamodel.LdaModel(corpus5, num_topics = NUM_TOPICS,
                                           id2word=dictionary5, passes=15)

#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics5 = ldamodel5.print_topics(num_words=20)

#위 내용을 출력
for topic in topics5:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.025*"배송" + 0.023*"네이버" + 0.015*"멤버십" + 0.014*"쿠팡" + 0.013*"서비스" + 0.012*"상품" + 0.012*"무료" + 0.010*"등" + 0.009*"보장" + 0.009*"도착" + 0.008*"고객" + 0.008*"반품" + 0.008*"할인" + 0.008*"마켓" + 0.008*"판매" + 0.007*"신세계" + 0.007*"수" + 0.007*"회원" + 0.007*"회비" + 0.007*"주문"')
(1, '0.017*"국내" + 0.014*"알리" + 0.014*"익스" + 0.014*"스" + 0.014*"프레" + 0.013*"커머스" + 0.011*"등" + 0.009*"것" + 0.009*"수" + 0.008*"한국" + 0.007*"중국" + 0.007*"매출" + 0.007*"테무" + 0.006*"위" + 0.006*"시장" + 0.006*"지난해" + 0.005*"고" + 0.005*"며" + 0.005*"경쟁" + 0.004*"이커머스"')
(2, '0.034*"쿠팡" + 0.018*"멤버십" + 0.016*"등" + 0.016*"인상" + 0.015*"회원" + 0.013*"것" + 0.012*"와우" + 0.011*"무료" + 0.011*"배송" + 0.011*"혜택" + 0.009*"요금" + 0.009*"서비스" + 0.008*"중국" + 0.008*"소비자" + 0.007*"가격" + 0.007*"배달" + 0.006*"월" + 0.006*"수" + 0.006*"알리" + 0.006*"국내"')


In [140]:
pyLDAvis.enable_notebook()
vis5 = pyLDAvis.gensim_models.prepare(ldamodel5, corpus5, dictionary5)
pyLDAvis.display(vis5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [144]:
coherence_values5 = []
model_list5 = []
iters5 = [5,6,7,8]

for num_topics in iters5:
     model5 = LdaModel(corpus=corpus5, id2word=dictionary5, num_topics=num_topics)
     model_list5.append(model5)
     coherencemodel5 = CoherenceModel(model=model5, texts=tokenized_data5, dictionary=dictionary5, coherence='c_v')
     coherence_values5.append(coherencemodel5.get_coherence())

for i in range(0, len(iters)):
  print(model_list5[i], coherence_values5[i])  #topic num별로 coherence 가 높은 것을 찾기

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LdaModel<num_terms=1791, num_topics=5, decay=0.5, chunksize=2000> 0.40122704918344204
LdaModel<num_terms=1791, num_topics=6, decay=0.5, chunksize=2000> 0.36731047669184486
LdaModel<num_terms=1791, num_topics=7, decay=0.5, chunksize=2000> 0.41840666024810735
LdaModel<num_terms=1791, num_topics=8, decay=0.5, chunksize=2000> 0.3729271149096494
